In [194]:
import pandas as pd
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso,LassoCV,LassoLarsCV
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

df = pd.read_csv('/Users/zhou/Downloads/data_y.csv',header = 0, names= ["asset","date",'lastprice','volume'])
asset_series = df['asset'].unique()
df['date'] = pd.to_datetime(df['date'])
date_list = df['date'].unique()
date_list.sort()
last_date = df.date.iloc[-1]
begin_date = df.date.iloc[0]
df_org = df.set_index(['asset','date'])
df

,asset,date,lastprice,volume
0,1332 JT,2013-01-04,169.0987,1464100
1,1332 JT,2013-01-07,166.3266,1783500
2,1332 JT,2013-01-08,166.3266,1759800
3,1332 JT,2013-01-09,165.4026,767800
4,1332 JT,2013-01-10,167.2507,1503100
...,...,...,...,...
451108,9984 JT,2021-03-15,10370.0000,12041200
451109,9984 JT,2021-03-16,10620.0000,11346000
451110,9984 JT,2021-03-17,10400.0000,9771000
451111,9984 JT,2021-03-18,10220.0000,13941600


In [195]:
def asset_list(index): 
    

IndentationError: expected an indented block (3720253323.py, line 2)

In [196]:
def one_asset(asset,df,trade_date):
#     df = pd.read_csv(index[i],header = 0, names= ["number","date",'high','lastprice','low','open','close','time','value','volume'])
    df['log1d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(1)))
    df['log2d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(2)))
    df['log3d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(3)))
    df['log5d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(5)))
    df['log10d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(10)))
    df['log20d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(20)))
    df['log30d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(30)))
    df['log60d']= pd.DataFrame(np.log(df.lastprice/df.lastprice.shift(60)))
        # ma_list = [5, 10, 20,30,40]
        # # 计算简单算术移动平均线MA - 注意：stock_data['close']为股票每天的收盘价
        # for ma in ma_list:
        #     df['ma' + str(ma*3)+'s'] = df['lastprice'].rolling(window=ma).mean()
        #     df['std' + str(ma*3)+'s'] = df['lastprice'].rolling(window=ma).std()
            #pd.rolling_mean(df.lastprice, ma)

        # for i in range(40,len(df.lastprice)):
        #     df['ma15s'].iloc[i] = np.mean(df['lastprice'].iloc[i-4:i+1])/df.lastprice.iloc[i] 
        #     df['ma30s'].iloc[i] = pd.DataFrame(np.mean(df['lastprice'].iloc[i-9:i+1])/df.lastprice.iloc[i])
        #     df['ma60s'].iloc[i] = np.mean(df['lastprice'].iloc[i-19:i+1])/df.lastprice.iloc[i]
        #     df['ma90s'].iloc[i] = pd.DataFrame(np.mean(df['lastprice'].iloc[i-29:i+1])/df.lastprice.iloc[i])  
        #     df['ma120s'].iloc[i] = pd.DataFrame(np.mean(df['lastprice'].iloc[i-39:i+1])/df.lastprice.iloc[i])
        #     df['v30s'].iloc[i] = pd.DataFrame(np.std(df['lastprice'].iloc[i-9:i+1]))
        #     df['v60s'].iloc[i] = pd.DataFrame(np.std(df['lastprice'].iloc[i-19:i+1]))
        #     df['v90s'].iloc[i] = pd.DataFrame(np.std(df['lastprice'].iloc[i-29:i+1]))
        #     df['v120s'].iloc[i] = pd.DataFrame(np.std(df['lastprice'].iloc[i-39:i+1]))      
    df['volume1d'] = df['volume']- df['volume'].shift(1)
    df['volume2d'] = df['volume']- df['volume'].shift(2)
    df['volume3d'] = df['volume']- df['volume'].shift(3)
    
    df['volume5d'] = df['volume']- df['volume'].shift(5)
    df['volume10d'] = df['volume']-df['volume'].shift(10)
    df['volume20d'] = df['volume']-df['volume'].shift(20)
    df['volume30d'] = df['volume']-df['volume'].shift(30)
    df['volume60d'] = df['volume']-df['volume'].shift(60)
    df['y'] = pd.DataFrame(np.log(df.lastprice.shift(-20)/df.lastprice))

    df_feature_pd = df[['lastprice','volume','log1d','log2d','log3d','log5d','log10d','log20d','log30d','log60d',
                      'volume1d','volume2d','volume3d','volume5d','volume10d','volume20d','volume30d','volume60d']]
#         df_feature_pd_witht = df[['lastprice','volume','log1d','log2d','log3d','log5d','log10d','log20d','log30d','log60d',
#                       'volume1d','volume2d','volume3d','volume5d','volume10d','volume20d','volume30d','volume60d']]
#         #df_feature_pd = df[['high','lastprice','low','open','close','volume','log3s','log6s','log9s','log15s','log30s','log60s','log90s','log120s',
#         #              'ma15s','ma30s','ma60s','ma90s','ma120s','std30s','std60s','std90s','std120s','(high-lastprice)/open','(lastprice-low)/open','low/open','high/open','(open-close)/close']]
    df_feature_train_pd = df_feature_pd[df.index <= trade_date].iloc[60::]
    df_feature_test_pd = df_feature_pd[df.index > trade_date].iloc[::-1].iloc[20::].iloc[::-1]
#         df_feature_train_pd_witht = df_feature_pd_witht[df.date <= '03/26/2019'].iloc[40::]
#         df_feature_test_pd_witht = df_feature_pd_witht[df.date > '03/26/2019'].iloc[::-1].iloc[20::].iloc[::-1]
#         train_xset_list.append(df_feature_train_pd_witht)
#         test_xset_list.append(df_feature_test_pd_witht)
    df_feature_train = df_feature_train_pd.values
    df_feature_test = df_feature_test_pd.values
    df_feature = df_feature_pd.values
    new_df_feature_train=df_feature_train/np.max(df_feature_train,axis = 0)
    #new_df_feature_test=df_feature_test_pd/np.max(df_feature_train_pd,axis = 0)
    new_df_feature_test=df_feature_test/np.max(df_feature_test,axis = 0)
        #arr_mean_y = np.mean(df_y) #求均值
        #arr_std_y = np.std(df_y,ddof=1) #求标准差
        #new_df_y=(df_y-arr_mean_y )/arr_std_y
    df_y_train = df['y'][df.index <= trade_date].iloc[60::]
#     df_y_test = df['y'][df.index > trade_date].iloc[::-1].iloc[20::].iloc[::-1]
#         df_y_train_witht = df[['y','date','time']][df.date <= '12/31/2020'].iloc[40::]
#         df_y_test_witht = df[['y','date','time']][df.date > '12/31/2020'].iloc[::-1].iloc[20::].iloc[::-1]
#         train_yset_list.append(df_y_train_witht) 
#         test_yset_list.append(df_y_test_witht)
    df_y_train = df_y_train.values
#     df_y_test = df_y_test.values
    new_df_y_train = df_y_train/np.max(df_y_train,axis = 0)
    #new_df_y_test = df_y_test/np.max(df_y_train,axis = 0)
#     new_df_y_test = df_y_test/np.max(df_y_test,axis = 0)


        # a = np.sort(new_df_y_train)
        # b = np.array(new_df_y_train)
        # for i in range(len(new_df_y_train)):
        #     b[i] = (i+1) / len(new_df_y_train)
        # print(a[int(len(new_df_y_train)*0.3)])
        # print(a[int(len(new_df_y_train)*0.7)])
        # plt.plot(a,b)
        # model = LassoCV()  # LassoCV自动调节alpha可以实现选择最佳的alpha。
        # #model = LassoLarsCV()  # LassoLarsCV自动调节alpha可以实现选择最佳的alpha
        # model.fit(new_df_feature_train,new_df_y_train)   # 线性回归建模
        # print('系数矩阵:\n',model.coef_)
        # #print('线性回归模型:\n',model)
        # #print('最佳的alpha：',model.alpha_) 
        # atest = np.sort(new_df_y_test)
        #print(atest[int(len(new_df_y_test)*0.3)])
        #print(atest[int(len(new_df_y_test)*0.7)])
#         df_feature_Lasso_pd_witht = df[["date","time",'high','lastprice','volume','log3s','log6s','log9s','log15s','log30s','log60s','log90s','log120s',
#                       '(high-lastprice)/open','(lastprice-low)/open','(open-close)/close']]
        #random forest test
#         bool_series = new_df_y_train > 0.005
#         #print(new_df_y_train.size)
#         y_train = pd.Series(np.zeros(new_df_y_train.size))
#         #bool_series.index = y_train.index
#         y_train[bool_series] = 1
#         bool_series = (new_df_y_train) < -0.005
#         #bool_series.index = y_train.index
#         y_train[bool_series] = -1

#         bool_series = (new_df_y_test) > 0.005
#         y_test = pd.Series(np.zeros(new_df_y_test.size))
#         #bool_series.index = y_test.index
#         #y = 0
#         y_test[bool_series] = 1
#         bool_series = (new_df_y_test) < -0.005
#         #bool_series.index = y_test.index
#         y_test[bool_series] = -1
#         train_yset_01_list.append(pd.DataFrame(y_train))
#         test_yset__01_list.append(pd.DataFrame(y_test))
        #print(y[100:900])
    clf = RandomForestRegressor(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
    clf.fit(new_df_feature_train, df_y_train, sample_weight=None)  
    scores = cross_val_score(clf, new_df_feature_train, df_y_train,cv=10)
#     score_train_list.append(clf.score(new_df_feature_train, df_y_train))
#     score_test_list.append(clf.score(new_df_feature_test, df_y_test))
#     scores_list.append(scores.mean())
            #print(clf.feature_importances_)
            #print(clf.score(new_df_feature_train, y_train))  
            #print(clf.score(new_df_feature_test, y_test))
        
            # clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
            # scores = cross_val_score(clf, new_df_feature, y)
            #print(scores.mean())
    predictions = clf.predict(new_df_feature_test)
    predictions = pd.DataFrame(predictions,index = df_feature_test_pd.index)
    price = pd.DataFrame(df_feature_test_pd.lastprice,index = df_feature_test_pd.index)
#     predictions_list.append(predictions)
    return predictions,scores,price


Parameter setting

In [197]:
start_date = pd.to_datetime('01/01/2021')
rebalance_period = 20  # 20天换仓
transaction_cost = 0.001  # 交易费为万分之10


Training: Use random forest to predict. Here, the trading frequancy is 20 trading days.

One: traning is not adaptive

In [198]:
pre_test = pd.DataFrame()
price_test = pd.DataFrame()

for i in asset_series:
    df = pd.DataFrame()
    df[['lastprice','volume']] = df_org.loc[i]
#     print(lat)
#     print(df.index[-1]==last_date)
    if (df.index[0]==begin_date)&(df.index[-1]==last_date):
        predictions,scores,price = one_asset(i,df,start_date)
        pre_test[i] = predictions
        price_test[i] = price

pre_test

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

...

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_test[i] = predictions
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  price_test[i] = price
/var/folders/50/85gxfbhs1lg3qzr_1ddn62n80000gn/T/ipykernel_67415/300010092.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

,1332 JT,1605 JT,1721 JT,1801 JT,1802 JT,1803 JT,1812 JT,1925 JT,1928 JT,1963 JT,...,9502 JT,9503 JT,9531 JT,9532 JT,9602 JT,9613 JT,9735 JT,9766 JT,9983 JT,9984 JT
date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,-0.018939,0.003076,0.008247,-0.027378,-0.075030,-0.116087,-0.026987,0.011689,0.003778,-0.026465,...,-0.020225,-0.013980,-0.082601,-0.046481,0.030419,-0.002494,0.041968,-0.069077,-0.136032,0.029471
2021-01-05,0.002930,-0.006106,-0.060357,-0.029978,-0.110961,-0.036406,-0.065880,0.018989,-0.055861,-0.028884,...,-0.025202,-0.020319,-0.077105,-0.043631,0.003488,0.024940,0.000022,-0.066285,-0.135481,0.039391
2021-01-06,-0.062118,-0.055911,-0.060585,-0.039414,-0.122187,-0.115944,0.006131,0.010083,-0.010172,-0.010369,...,-0.010587,-0.032077,-0.097532,-0.049600,0.005532,0.060582,0.033660,-0.048928,-0.029593,0.058375
2021-01-07,-0.040547,-0.035640,-0.047727,-0.068679,-0.090097,-0.146940,-0.046812,-0.033392,-0.010785,0.036078,...,-0.022524,-0.044527,-0.084286,-0.060937,0.013477,-0.064922,0.000830,-0.024479,-0.072134,0.067698
2021-01-08,0.011170,-0.031788,-0.045364,-0.153280,-0.111649,-0.147236,-0.055404,-0.003504,-0.009088,0.070912,...,-0.048316,-0.057901,-0.122016,-0.056297,-0.044304,-0.050745,-0.012803,0.023852,-0.167594,0.066568
2021-01-12,0.018327,-0.054434,-0.033760,-0.145032,-0.101840,-0.138109,-0.054515,-0.062608,-0.001871,0.004664,...,-0.097554,-0.076369,-0.094036,-0.110846,0.062038,-0.051137,-0.002323,0.061398,-0.069977,-0.015520
2021-01-13,0.025242,-0.045809,-0.022087,-0.121981,-0.102348,-0.197406,-0.046977,-0.059349,-0.002847,0.033383,...,-0.072513,-0.048337,-0.096683,-0.045818,-0.010623,-0.101832,-0.023574,0.081042,-0.126414,0.026531
2021-01-14,-0.001391,0.002470,-0.049412,-0.137831,-0.038430,-0.112005,-0.047107,-0.034325,-0.071045,0.010123,...,-0.060910,-0.060857,-0.098281,0.003828,-0.020077,-0.073202,-0.027342,0.064923,-0.144499,-0.050415
2021-01-15,-0.016707,-0.095565,-0.025274,-0.133647,-0.071924,-0.114405,-0.044122,-0.049489,-0.034384,0.015962,...,-0.079636,-0.090742,-0.028470,-0.013592,-0.023055,-0.038585,-0.023277,0.029088,0.041788,0.002161


Two: traning is adaptive(Need to run for long time, have not tested yet)

In [ ]:
for i, date in enumerate(date_list):
    if date > start_date:
        index = i
        break
pre_test = pd.DataFrame()
price_test = pd.DataFrame()
for date_index in range(index,len(date_list)-20,rebalance_period):
    trade_date = date_list[date_index]
    print(date_index-index)
    for i in asset_series:
        df = pd.DataFrame()
        df[['lastprice','volume']] = df_org.loc[i]
    #     print(lat)
    #     print(df.index[-1]==last_date)
        if (df.index[0]==begin_date)&(df.index[-1]==last_date):
            predictions,scores,price = one_asset(i,df,trade_date)
            pre_test[i] = predictions[0:20]
            price_test[i] = price[0:20]
        
pre_test

0


Backtest: Choose the asset with highest scores to construct a equal weight portfolio. The transaction cost is not accurately calcucated

In [207]:

df_grade = pre_test
df_price = price_test
# 初始化一个空的DataFrame来存储投资组合和收益
portfolio_return = []
portfolio = None  # 用于存储上一个周期的投资组合

# 设置参数


# 开始执行策略
for i in range(0, len(df_grade), rebalance_period):
    # 选择当前周期的结束日期
    period_end = min(i + rebalance_period, len(df_grade))
    
    # 对当前周期内的grade排序并选择前十
    top_assets = df_grade.iloc[i:period_end].mean().nlargest(10).index
    
    # 如果现有投资组合存在，计算交易费用
    if portfolio is not None:
        # 计算需要卖出的部分（不在新的前十名中的资产）
        sell_assets = portfolio.difference(top_assets)
        # 计算需要买入的部分（在新的前十名但不在旧组合中的资产）
        buy_assets = top_assets.difference(portfolio)
    else:
        # 如果是第一个周期，购买所有的前十名资产
        buy_assets = top_assets
        sell_assets = []
    
#     计算交易费用
#     transaction_costs = df_price.iloc[i:period_end][buy_assets].iloc[0] * transaction_cost
#     total_transaction_costs = transaction_costs.sum() * (1/10)
#     print(total_transaction_costs)
#     print(len(buy_assets))
    # 更新投资组合
    portfolio = top_assets
    
    # 计算每个资产的收益，并考虑交易费
    asset_returns = (df_price.iloc[period_end-1, df_price.columns.get_indexer(top_assets)] 
                     / df_price.iloc[i, df_price.columns.get_indexer(top_assets)] - 1) / 10
    
    # 计算总收益
    period_return = asset_returns.sum()/(1+len(buy_assets)*transaction_cost/10)
    
    # 将收益分配到相应的日期
    portfolio_return.append(period_return+1)
# print(portfolio_return)
# 计算累积收益
portfolio_return_cumulative = np.array(portfolio_return).cumprod()-1
cumulative_max = (1 + pd.DataFrame(portfolio_return_cumulative)).cummax()
drawdown = (1 + portfolio_return_cumulative) - np.array(cumulative_max)
max_drawdown = drawdown.min()

# 计算波动率
volatility = np.array(portfolio_return).std() * np.sqrt(252/20)

# 打印投资组合的累积收益
print(portfolio_return_cumulative)


[0.0529475  0.07474049]
